In [3]:
import dash
from dash.dependencies import Input, Output
from dash import dcc
from dash import html
from pandas_datareader import data as web
#from datetime import datetime as dt
import plotly.graph_objs as go
#import dash_core_components as dcc
#import dash_html_components as html
import pandas as pd
import json
import plotly.express as px
import dash_table as dt
import plotly.figure_factory as ff
import numpy as np
data_long_new = pd.read_csv('model_results_merge.csv')

In [4]:
app = dash.Dash()
metric_filter = data_long_new['variable'].unique()
method_filter = data_long_new['Method'].unique()
model_filter = data_long_new['Model'].unique()
app.layout = html.Div([
    html.Div([
        html.Div([
                html.Div([      
                    
                    html.H2('Dashboard1: Performance of Classification Model by Methods and Metrics', style=dict(color='#7F90AC')),
                    ], className = "nine columns padded" )
            ], className = 'row gs-header gs-text-header'),
       html.Div([

        html.Div([
            dcc.Dropdown(
                id='metric_filter',
                options=[{'label': i, 'value': i} for i in metric_filter],
                value='Accuracy'
            )
        ],style={'width': '40%', 'display': 'inline-block','align-items': 'center','justify-content': 'center','padding': '0 20'}),
        html.Div([
            dcc.Dropdown(
                id='method_filter',
                options=[{'label': i, 'value': i} for i in method_filter],
                value='Dataset after Deleting all the NAs'
            )
        ],
        style={'width': '40%', 'display': 'inline-block','align-items': 'center','justify-content': 'center'})
    ]),
    html.Div([
        html.Div([
        dcc.Graph(
            id='method-filter-scatter',
            hoverData={'points': [{'customdata': 'Ridge Classifier'}]}
        )],style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'}),
        html.Div([
        dcc.Graph(
            id = 'confusion_matrix1')
    ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 20'})
    ])
    ],className = 'page'),
    # Page 2
     html.Div([
        html.Div([
                html.Div([      
                    
                    html.H2('Dashboard2: Performance of Classification Model by Models and Metrics', style=dict(color='#7F90AC')),
                    ], className = "nine columns padded" )
            ], className = 'row gs-header gs-text-header'),
       html.Div([

        html.Div([
            dcc.Dropdown(
                id='metric_filter1',
                options=[{'label': i, 'value': i} for i in metric_filter],
                value='Accuracy'
            ),
            dcc.Dropdown(
                id='model_filter',
                options=[{'label': i, 'value': i} for i in model_filter],
                value='Ridge Classifier',
            )
        ],
        style={'width': '40%', 'display': 'inline-block'})
    ]),
    html.Div([
        html.Div([
        dcc.Graph(
            id='model-filter-scatter',
            hoverData={'points': [{'customdata':'Dataset after Deleting all the NAs'}]}
        )
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 25','align-items': 'center','justify-content': 'center'}),
        html.Div([
        dcc.Graph(
            id='confusion_matrix'
        )
        ], style={'width': '49%', 'display': 'inline-block', 'padding': '0 25', 'align-items': 'center','justify-content': 'center'})
    ])
    ],className = 'page'),
])
@app.callback(
    dash.dependencies.Output('method-filter-scatter', 'figure'),
    [dash.dependencies.Input('method_filter', 'value'),
     dash.dependencies.Input('metric_filter','value')]
    ) 
def update_graph1(selected_methods,selected_metrics):
    filtered_df = data_long_new[(data_long_new['variable'] == selected_metrics) & (data_long_new['Method'] == selected_methods)]
    return {
        'data':[go.Scatter(
           x = filtered_df['Model'],
           y = filtered_df['value'],
           text = filtered_df['Model'],
           customdata = filtered_df['Model']
           #marker_color=colors
        )],
        'layout': go.Layout(
           
           yaxis={
               'title': 'Score'
           },
           height = 450,
           hovermode = 'closest',
           title = '<b>Performance of Classifier<b>'
        )
    }

def create_cm(dff):
    dff = dff.drop(['Method','Model','variable','value'],axis = 1)
    arr = np.zeros((2, 2), dtype=np.int)
    matrix = dff.values
    arr[0,0] = matrix[0,0]
    arr[0,1] = matrix[0,1]
    arr[1,0] = matrix[0,2]
    arr[1,1] = matrix[0,3]
    x = ['deceased','discharged']
    y = ['deceased','discharged']
    z_text = [[str(y) for y in x] for x in arr]
    fig = ff.create_annotated_heatmap(arr, x=x, y=y, annotation_text=z_text, colorscale=['aliceblue','aqua','aquamarine','darkturquoise'])
    fig.update_layout(title_text='<i><b>Confusion matrix</b></i>',
                  #xaxis = dict(title='x'),
                  #yaxis = dict(title='x')
                 )
    return fig
@app.callback(
    dash.dependencies.Output('confusion_matrix1', 'figure'),
    [dash.dependencies.Input('method_filter', 'value'),
     dash.dependencies.Input('method-filter-scatter','hoverData')
    ]) 
def update_cm_graph(selected_methods,hoverData):
    model_name = hoverData['points'][0]['customdata']
    dff = data_long_new[(data_long_new['Model'] == model_name) & (data_long_new['Method'] == selected_methods)]
    return create_cm(dff)
    
@app.callback(
    dash.dependencies.Output('model-filter-scatter', 'figure'),
    [dash.dependencies.Input('model_filter', 'value'),
     dash.dependencies.Input('metric_filter1','value')]
    ) 
def update_graph(selected_models,selected_metrics):
    filtered_df = data_long_new[(data_long_new['variable'] == selected_metrics) & (data_long_new['Model'] == selected_models)]
    return {
        'data':[go.Scatter(
           x = filtered_df['Method'],
           y = filtered_df['value'],
           text = filtered_df['Method'],
           customdata = filtered_df['Method']
           #marker_color=colors
        )],
        'layout': go.Layout(
           
           yaxis={
               'title': 'Data_Preprocessing_Method '
           },
           height = 450,
           hovermode = 'closest',
           title = '<b>Performance of Data Preprocessing Method<b>'
        )
    }
@app.callback(
    dash.dependencies.Output('confusion_matrix', 'figure'),
    [dash.dependencies.Input('model_filter', 'value'),
     dash.dependencies.Input('model-filter-scatter','hoverData')
    ]) 
def update_cm_graph(selected_models,hoverData):
    model_name = hoverData['points'][0]['customdata']
    dff = data_long_new[(data_long_new['Method'] == model_name) & (data_long_new['Model'] == selected_models)]
    return create_cm(dff)
    
app.css.append_css({
    'external_url': 'https://codepen.io/chriddyp/pen/bWLwgP.css'
})

if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
C:\Users\lucylin\Anaconda3\lib\site-packages\dash\resources.py:72: UserWarning:

You have set your config to `serve_locally=True` but A local version of https://codepen.io/chriddyp/pen/bWLwgP.css is not available.
If you added this file with `app.scripts.append_script` or `app.css.append_css`, use `external_scripts` or `external_stylesheets` instead.
See https://dash.plotly.com/external-resources

127.0.0.1 - - [22/Nov/2021 21:28:50] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 21:28:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 21:28:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 21:28:51] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 21:28:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [22/Nov/2021 21:28:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0